참고: https://github.com/KerasKorea/KEKOxTutorial/blob/master/16_Ensembling%20ConvNets%20using%20Keras.md

In [2]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train / 255. 
x_test = x_test / 255. 

y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [4]:
input_shape = x_train[0, :, :, :].shape
# input_shape
model_input = tf.keras.layers.Input(shape=input_shape)

In [5]:
def conv_pool_cnn(model_input):
    
    x = tf.keras.layers.Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = tf.keras.layers.Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = tf.keras.layers.Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = tf.keras.layers.Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = tf.keras.layers.Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = tf.keras.layers.Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = tf.keras.layers.Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = tf.keras.layers.Conv2D(192, (1, 1), activation='relu')(x)
    x = tf.keras.layers.Conv2D(10, (1, 1))(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Activation(activation='softmax')(x)
    
    model = tf.keras.Model(model_input, x, name='conv_pool_cnn')
    
    return model

In [6]:
conv_pool_model = conv_pool_cnn(model_input)

In [7]:
conv_pool_model.summary() 

Model: "conv_pool_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 96)        2688      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 96)        83040     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 96)        83040     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 96)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 192)       166080    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 192)     

In [ ]:
def compile_and_train(model, num_epochs): 
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(), 
                  metrics=['accuracy']
                  )
    
    filepath = 'weights/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', period=1)
    tensor_board = tf.keras.callbacks.TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    hist = model.fit(x_train, y_train, batch_size=32, epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
    
    return hist

In [ ]:
_ = compile_and_train(conv_pool_model, num_epochs=1)

   1/1250 [..............................] - ETA: 22:03 - loss: 2.3006 - accuracy: 0.1562

   2/1250 [..............................] - ETA: 8:44 - loss: 2.3258 - accuracy: 0.1094 

   3/1250 [..............................] - ETA: 10:46 - loss: 2.3181 - accuracy: 0.1146

   4/1250 [..............................] - ETA: 9:15 - loss: 2.3188 - accuracy: 0.1094 

   5/1250 [..............................] - ETA: 8:30 - loss: 2.3168 - accuracy: 0.0938

...

 744/1250 [================>.............] - ETA: 2:32 - loss: 1.9232 - accuracy: 0.2530

 745/1250 [================>.............] - ETA: 2:31 - loss: 1.9231 - accuracy: 0.2531

 746/1250 [================>.............] - ETA: 2:31 - loss: 1.9233 - accuracy: 0.2530

 747/1250 [================>.............] - ETA: 2:31 - loss: 1.9226 - accuracy: 0.2533

 748/1250 [================>.............] - ETA: 2:30 - loss: 1.9221 - accuracy: 0.2536

In [ ]:
def evaluate_error(model): 
    pred = model.predict(x_test, batch_size=32)
    pred = np.argmax(pred, axis=1) 
    pred = np.expand_dims(pred, axis=1) 
    error = np.sum(np.not_equal(pred, y_test)) / y_test.shape[0]
    
    return error 

evaluate_error(conv_pool_model)

In [16]:
def all_cnn(model_input):
    
    x = tf.keras.layers.Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = tf.keras.layers.Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = tf.keras.layers.Conv2D(96, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = tf.keras.layers.Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = tf.keras.layers.Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = tf.keras.layers.Conv2D(192, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = tf.keras.layers.Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = tf.keras.layers.Conv2D(192, (1, 1), activation='relu')(x)
    x = tf.keras.layers.Conv2D(10, (1, 1))(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Activation(activation='softmax')(x)
        
    model = tf.keras.Model(model_input, x, name='all_cnn')
    
    return model

In [17]:
all_cnn_model = all_cnn(model_input)
all_cnn_model.summary() 

Model: "all_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 96)        2688      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 96)        83040     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 192)       166080    
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 16, 16, 192)       331968    
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 8, 8, 192)         3319

In [18]:
_ = compile_and_train(all_cnn_model, num_epochs=1)

1250/1250 [==============================] - 337s 270ms/step - loss: 1.9535 - accuracy: 0.2507 - val_loss: 1.7100 - val_accuracy: 0.3629


In [19]:
def nin_cnn(model_input):
    
    #mlpconv block 1
    x = tf.keras.layers.Conv2D(32, (5, 5), activation='relu',padding='valid')(model_input)
    x = tf.keras.layers.Conv2D(32, (1, 1), activation='relu')(x)
    x = tf.keras.layers.Conv2D(32, (1, 1), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    
    #mlpconv block2
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu',padding='valid')(x)
    x = tf.keras.layers.Conv2D(64, (1, 1), activation='relu')(x)
    x = tf.keras.layers.Conv2D(64, (1, 1), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    
    #mlpconv block3
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu',padding='valid')(x)
    x = tf.keras.layers.Conv2D(32, (1, 1), activation='relu')(x)
    x = tf.keras.layers.Conv2D(10, (1, 1))(x)
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Activation(activation='softmax')(x)
    
    model = tf.keras.Model(model_input, x, name='nin_cnn')
    
    return model


In [20]:

nin_cnn_model = nin_cnn(model_input)

In [22]:
nin_cnn_model.summary()

Model: "nin_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 28, 28, 32)        1056      
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 28, 28, 32)        1056      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 12, 12, 64)        1849

In [23]:
_ = compile_and_train(nin_cnn_model, num_epochs=1)


1250/1250 [==============================] - 49s 39ms/step - loss: 1.8952 - accuracy: 0.2774 - val_loss: 1.6510 - val_accuracy: 0.3892


In [24]:
conv_pool_model.load_weights('weights/conv_pool_cnn.01-1.44.hdf5')
all_cnn_model.load_weights('weights/all_cnn.01-1.95.hdf5')
nin_cnn_model.load_weights('weights/nin_cnn.01-1.90.hdf5')

In [25]:
models = [conv_pool_model, all_cnn_model, nin_cnn_model]

In [26]:
def ensemble(models, model_input): 
    outputs = [model.outputs[0] for model in models]
    y = tf.keras.layers.Average()(outputs) 
    
    model = tf.keras.Model(model_input, y)
    
    return model 

In [27]:
ensemble_model = ensemble(models, model_input)

In [28]:
ensemble_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 28, 28, 32)   2432        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 28, 28, 32)   1056        conv2d_18[0][0]                  
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 96)   2688        input_1[0][0]                    
______________________________________________________________________________________________

In [29]:
evaluate_error(ensemble_model)


9.1513

In [30]:
pair_a = [conv_pool_model, all_cnn_model]
pair_b = [conv_pool_model, nin_cnn_model]
pair_c = [all_cnn_model, nin_cnn_model]

pair_a_ensemble_model = ensemble(pair_a, model_input)

In [31]:
evaluate_error(pair_a_ensemble_model)

9.2126

In [32]:
pair_b_ensemble_model = ensemble(pair_b, model_input)
evaluate_error(pair_b_ensemble_model)

9.0938

In [33]:
pair_c_ensemble_model = ensemble(pair_c, model_input)
evaluate_error(pair_c_ensemble_model)


9.1809

# Ensemble with voting

In [34]:
voting_models = [] 

voting_models.append(conv_pool_model)
voting_models.append(all_cnn_model)
voting_models.append(nin_cnn_model)

In [36]:
# voting_models = [] 
learnt_voting_models = [] 
for i in range(3):
    voting_models[i].fit(x_train, y_train, epochs=2,
                         steps_per_epoch=x_train.shape[0] // 32, 
                         validation_data = (x_test, y_test),
                         callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=3, factor=0.1)],
                         verbose=2
                         )
    
    learnt_voting_models.append(voting_models[i])
    

Epoch 1/2
1562/1562 - 558s - loss: 1.0791 - accuracy: 0.6091 - val_loss: 0.9659 - val_accuracy: 0.6570
Epoch 2/2
1562/1562 - 519s - loss: 0.8538 - accuracy: 0.6986 - val_loss: 0.7870 - val_accuracy: 0.7300
Epoch 1/2
1562/1562 - 404s - loss: 1.4591 - accuracy: 0.4587 - val_loss: 1.3612 - val_accuracy: 0.5045
Epoch 2/2
1562/1562 - 377s - loss: 1.1884 - accuracy: 0.5699 - val_loss: 1.1036 - val_accuracy: 0.6105
Epoch 1/2
1562/1562 - 53s - loss: 1.5807 - accuracy: 0.4153 - val_loss: 1.4502 - val_accuracy: 0.4736
Epoch 2/2
1562/1562 - 47s - loss: 1.4369 - accuracy: 0.4751 - val_loss: 1.3747 - val_accuracy: 0.5075


In [37]:
voting_labels = [] 

for m in voting_models:
    prediction = np.argmax(m.predict(x_test), axis=1)
    voting_labels.append(prediction)


In [38]:
voting_labels

[array([3, 1, 8, ..., 5, 0, 7], dtype=int64),
 array([3, 1, 8, ..., 5, 6, 7], dtype=int64),
 array([3, 1, 8, ..., 5, 4, 4], dtype=int64)]

In [39]:
voting_labels = np.array(voting_labels)
voting_labels

array([[3, 1, 8, ..., 5, 0, 7],
       [3, 1, 8, ..., 5, 6, 7],
       [3, 1, 8, ..., 5, 4, 4]], dtype=int64)